In [1]:
import sys
sys.path.append('..')
import requests
import json
import os
from ipywidgets import Textarea, VBox, Button, Output, IntProgress, Layout, Dropdown, HTML, Accordion
from IPython.display import display, clear_output
import time
from src.core import (
    process_order_request, 
    process_ride_plan_request, 
    process_event_plan_request,
    load_state,
    save_state
)

# This notebook now relies on src/core.py to manage the state file.
# The faulty initialization logic has been removed to fix the duplicate entry bug.

# --- Define User Queries for All Domains ---
tax_queries = [
    "We had two burgers at 10 euros each and a large soda for 3 euros."
]
cycling_queries = [
    "I want to plan a long, flat ride. Let's say 100 kilometers."
]
event_queries = [
    "Please create a new venue called 'Grand Ballroom'. It has a capacity of 500 and comes with an A/V system.",
    "I need to schedule a session called 'Intro to Transformers' hosted by Dr. Sharma. Please book it in the 'Grand Ballroom'. We expect 450 people, and it requires AV.",
    "Let's plan our AI Summit. We need a 'Deep Dive into NLP' workshop in Workshop Room A, which has a capacity of 40 but no A/V system. The workshop requires A/V and will have 35 attendees."
]
print("✅ System ready. Three domains are configured.")

✅ System ready. Three domains are configured.


In [2]:
def get_available_models():
    """Fetches a list of available models from the Ollama API."""
    try:
        response = requests.get('http://localhost:11434/api/tags')
        response.raise_for_status()
        models = response.json().get('models', [])
        return [model['name'] for model in models]
    except requests.exceptions.RequestException:
        return []

def create_multi_domain_ui():
    available_models = get_available_models()
    if not available_models:
        display(HTML("<p style='color:red; font-weight:bold;'>ERROR: Could not connect to Ollama. Please ensure it is running and you have pulled at least one model (e.g., 'ollama pull llama3:8b').</p>"))
        return

    # --- UI Widget Definitions ---
    model_selector = Dropdown(options=available_models, value=available_models[0] if available_models else None, description='Select Model:', style={'description_width': 'initial'})
    domain_selector = Dropdown(options=['Tax Calculation', 'Cycling Ride Planner', 'Event Planning'], value='Event Planning', description='Select Domain:', style={'description_width': 'initial'})
    role_selector = Dropdown(options=['admin', 'scheduler'], value='admin', description='Select Role:', style={'description_width': 'initial'}, layout={'visibility': 'hidden'})
    query_input = Textarea(value='', description='User Query:', style={'description_width': 'initial'}, layout={'width': '95%', 'height': '100px'})
    run_button = Button(description='Generate DSL & Execute', button_style='success', icon='play')
    progress_bar = IntProgress(value=0, min=0, max=10, description='Waiting...', bar_style='info', orientation='horizontal', layout={'visibility': 'hidden'})
    output_area = Output()
    
    # --- New State Display Widget ---
    state_html_view = HTML(value="Select the Event Planning domain to see the state.")
    state_accordion = Accordion(children=[state_html_view], titles=('Current Conference State',))
    state_accordion.selected_index = None # Start collapsed
    state_accordion.layout.visibility = 'hidden'

    # --- State Display Logic ---
    def update_state_display():
        """Reads state.json and renders it as HTML tables inside the accordion."""
        try:
            current_state = load_state()
            venues = current_state.get('venues', {})
            sessions = current_state.get('sessions', [])
            
            if not venues and not sessions:
                state_html_view.value = "<p><i>No venues or sessions have been created yet.</i></p>"
                return

            # Venue Table
            venues_html = "<h4>Venues</h4><table border='1' style='width:100%; border-collapse: collapse;'><tr><th style='padding: 5px; text-align: left;'>Name</th><th style='padding: 5px; text-align: left;'>Capacity</th><th style='padding: 5px; text-align: left;'>Has A/V</th></tr>"
            for name, props in venues.items():
                venues_html += f"<tr><td style='padding: 5px;'>{name}</td><td style='padding: 5px;'>{props.get('capacity', 'N/A')}</td><td style='padding: 5px;'>{props.get('has_av_system', 'N/A')}</td></tr>"
            venues_html += "</table>"
            
            # Session Table
            sessions_html = "<h4 style='margin-top: 15px;'>Scheduled Sessions</h4><table border='1' style='width:100%; border-collapse: collapse;'><tr><th style='padding: 5px; text-align: left;'>Name</th><th style='padding: 5px; text-align: left;'>Venue</th><th style='padding: 5px; text-align: left;'>Host</th><th style='padding: 5px; text-align: left;'>Attendees</th></tr>"
            for session in sessions:
                 sessions_html += f"<tr><td style='padding: 5px;'>{session.get('name', 'N/A')}</td><td style='padding: 5px;'>{session.get('in_venue', 'N/A')}</td><td style='padding: 5px;'>{session.get('hosted_by', 'N/A')}</td><td style='padding: 5px;'>{session.get('expected_attendees', 'N/A')}</td></tr>"
            sessions_html += "</table>"

            state_html_view.value = venues_html + sessions_html
        except Exception as e:
            state_html_view.value = f"<p style='color:red;'>Error loading state: {e}</p>"

    # --- Event Handlers ---
    def on_domain_change(change):
        domain = change['new']
        if domain == 'Tax Calculation': 
            query_input.value = tax_queries[0]
            role_selector.layout.visibility = 'hidden'
            state_accordion.layout.visibility = 'hidden'
        elif domain == 'Cycling Ride Planner': 
            query_input.value = cycling_queries[0]
            role_selector.layout.visibility = 'hidden'
            state_accordion.layout.visibility = 'hidden'
        elif domain == 'Event Planning': 
            query_input.value = event_queries[0]
            role_selector.layout.visibility = 'visible'
            state_accordion.layout.visibility = 'visible'
            update_state_display()
    domain_selector.observe(on_domain_change, names='value')

    def run_analysis(button):
        with output_area:
            clear_output(wait=True)
            run_button.disabled = True
            progress_bar.description = 'LLM processing...'; progress_bar.layout.visibility = 'visible'; progress_bar.value = 5

            selected_model = model_selector.value
            domain = domain_selector.value
            query = query_input.value
            selected_role = role_selector.value
            
            result = {}
            if domain == 'Tax Calculation': result = process_order_request(query, model_name=selected_model)
            elif domain == 'Cycling Ride Planner': result = process_ride_plan_request(query, model_name=selected_model)
            elif domain == 'Event Planning': result = process_event_plan_request(query, selected_role, model_name=selected_model)

            progress_bar.value = 10
            
            html_parts = []
            if result.get('llm_generated_dsl'):
                html_parts.append("<h4>1. LLM-Generated DSL Code</h4>")
                html_parts.append(f"<pre style='background-color:#f5f5f5; border: 1px solid #ccc; padding: 10px; border-radius: 5px;'>{result['llm_generated_dsl']}</pre>")

            html_parts.append("<h4>2. Interpreter Execution Result</h4>")
            if result["status"] == "success":
                dsl_result = result['interpreter_result']
                if domain == 'Event Planning':
                    summary_html = f"<b>✅ {dsl_result['message']}</b>"
                    html_parts.append(summary_html)
                    update_state_display() # Refresh the state view on success
                else:
                     html_parts.append(f"<p style='color:green; font-weight:bold;'>Validation Successful!</p><pre>{dsl_result}</pre>")
            else:
                html_parts.append(f"<h4 style='color:red;'>🔴 PROCESSING ERROR</h4><p>{result['message']}</p>")
            
            display(HTML(''.join(html_parts)))

            run_button.disabled = False
            progress_bar.layout.visibility = 'hidden'
            progress_bar.value = 0
            
    run_button.on_click(run_analysis)
    
    # --- Initial UI Setup ---
    print("--- Multi-Domain DSL Validator ---")
    on_domain_change({'new': domain_selector.value})
    display(VBox([model_selector, domain_selector, role_selector, query_input, run_button, progress_bar, state_accordion, output_area]))

create_multi_domain_ui()

--- Multi-Domain DSL Validator ---
